In [2]:
import pandas as pd
import numpy as np

In [18]:
autoregression = pd.read_csv("../outputs/predictions/sequifier-default-best-1575-all-autoregression-predictions.csv")
onestep = pd.read_csv("../outputs/predictions/sequifier-default-best-1575-all-onestep-predictions.csv")

In [19]:
target = pd.read_csv("../data/exchange_rate.txt").drop(columns=["itemPosition"])

In [20]:
columns = ["Australia", "British", "Canada", "Switzerland", "China", "Japan", "NewZealand", "Singapore"]

In [47]:
np.mean(np.abs(autoregression[columns].values - target[columns].tail(738).values))

0.10851065223804703

In [83]:
np.mean(np.abs( (np.tile(target[columns].tail(739).head(1).values, 738)).reshape(738, len(columns)) - target[columns].tail(738).values))

0.052383750169376686

In [84]:
np.mean(np.abs(onestep[columns].values - target[columns].tail(738).values))

0.023372470950462715

In [86]:
np.mean(np.abs(target[columns].tail(739).head(738).values - target[columns].tail(738).values))

0.002519212059620596